In [ ]:
import pandas as pd
import re
from collections import Counter

In [ ]:
# read excell shet to df
import pandas as pd

path = "./Ocean Governance and ocean economy governance matrix_IGOs.xlsx"
data = pd.read_excel(path, sheet_name="Sheet1")

# backup
df = data.copy(deep=True)

In [ ]:
df.head()

In [ ]:
df = df[["Institutions", "Vertical"]]

In [ ]:
df.head()

In [ ]:
# Step 1: Cleaning the Data
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    # Correct typos and standardize terms
    text = text.replace("United Nations", "UN").replace("member states", "Member States").replace("OCean Decasde", "Ocean Decade")
    # Simplify verbose phrases
    text = re.sub(r'which are responsible for implementing', 'tasked with implementing', text)
    text = re.sub(r'including the UN', 'including UN', text)
    return text.strip()

df["Cleaned Practical Vertical Coordination"] = df["Vertical"].apply(clean_text)

# Step 2: Summarizing the Data
def summarize_text(text):
    summary = ""
    # Extract collaboration entities
    if "collaboration" in text.lower():
        summary += "Coordinates with "
        if "UN" in text: summary += "UN agencies, "
        if "Member States" in text: summary += "Member States, "
        if "national governments" in text or "governments" in text: summary += "governments, "
        summary = summary.rstrip(", ")
    # Extract governance structures
    if any(x in text for x in ["Assembly", "Council", "Secretariat", "Conference", "COP"]):
        summary += " through its "
        if "Assembly" in text: summary += "Assembly, "
        if "Council" in text: summary += "Council, "
        if "Secretariat" in text: summary += "Secretariat, "
        if "Conference" in text: summary += "Conference, "
        if "COP" in text: summary += "COP, "
        summary = summary.rstrip(", ")
    # Add regional or technical elements
    if "regional" in text.lower(): summary += " using regional structures"
    if any(x in text.lower() for x in ["technical", "assistance", "training"]): summary += " with technical support"
    return summary.strip()

df["Summarized Vertical Coordination"] = df["Cleaned Practical Vertical Coordination"].apply(summarize_text)

# Step 3: Assigning Themes
def assign_themes(summary):
    themes = []
    # Theme assignment rules
    if any(x in summary.lower() for x in ["assembly", "council", "secretariat", "conference", "cop"]):
        themes.append("Multi-Level Governance Structures")
    if any(x in summary.lower() for x in ["un agencies", "member states", "governments"]):
        themes.append("Collaboration with External Entities")
    if "regional" in summary.lower():
        themes.append("Regional and Local Implementation")
    if "technical" in summary.lower():
        themes.append("Technical and Capacity-Building Support")
    if any(x in summary.lower() for x in ["director", "secretary", "leadership"]):
        themes.append("Strategic Leadership and Oversight")
    return ", ".join(themes)

# Frequency analysis for theme validation
def validate_themes(df):
    all_themes = df["Summarized Vertical Coordination"].apply(assign_themes).str.split(", ").explode()
    theme_counts = Counter(all_themes)
    print("Theme Frequency Counts:", theme_counts)

df["Themes"] = df["Summarized Vertical Coordination"].apply(assign_themes)
validate_themes(df)


In [ ]:
df.head()

In [ ]:
# # Step 4: Output Table
# print("\nFinal Table:")
# print(df[["Institutions", "Cleaned Practical Vertical Coordination", "Summarized Vertical Coordination", "Themes"]].to_string(index=False))

# Save to CSV
df.to_csv("vertical_coordination_analysis.csv", index=False)
print("\nData saved to 'vertical_coordination_analysis.csv'")